### RFM
- Nessa etapa usa somente os dados relacionados as vendas.
- Primeiro iremos retirar as informações de Recência, frequência e monetario dos dados relacionados a vendas.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Importação dos dados (CSV) 

- Temporário até ter o banco de dados mysql pronto com os dados

In [2]:
dados_vendas = r"C:\Users\heloi\OneDrive\Desktop\Projeto - Inlytics\Dataset\vendas_info_limpo.csv"

df_vendas = pd.read_csv(dados_vendas)

### Dados limpos pós transformação ETL (começar daqui)

In [3]:
df_vendas.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


**Abaixo estamos:**

- Fazendo o ajuste do tipo de dado relacionado à data
- Criando o valor total da compra levando em consideração que os dados estavam separados
- Criando uma variável que contém a data de referencia com base nos dados do dataset.
- Calculando o RFM
- Criando os Scores para R, F, M (utilizando quintis)
- Criamos a função para seguimentarmos os clientes com base nos Scores de R, F, M
- Aplicamos a função no dataframe

In [4]:

df_vendas['InvoiceDate'] = pd.to_datetime(df_vendas['InvoiceDate'])

# Remover linhas com CustomerID nulo
df_vendas = df_vendas[df_vendas['CustomerID'].notnull()]

# Remover transações com Quantity <= 0 (cancelamentos ou erros)
df_vendas = df_vendas[df_vendas['Quantity'] > 0]

# Calcular valor total por linha
df_vendas['TotalPrice'] = df_vendas['Quantity'] * df_vendas['UnitPrice']

# 2. Definir data de referência para Recência (um dia após o último InvoiceDate)
data_ref = df_vendas['InvoiceDate'].max() + pd.Timedelta(days=1)

# 3. Calcular RFM
rfm = df_vendas.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (data_ref - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalPrice': 'sum'
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# 4. Criar scores R, F, M (de 1 a 5) usando quintis
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1]).astype(int)  # Recência: menor valor melhor (mais recente)
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)  # Frequência: maior melhor
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5]).astype(int)  # Monetário: maior melhor


# Função para segmentar clientes com base em R, F e M
def rfm_segment(row):
    if row['R_Score'] >= 4 and row['F_Score'] >= 4 and row['M_Score'] >= 4:
        return 'Champions'
    elif row['R_Score'] >= 3 and row['F_Score'] >= 3 and row['M_Score'] >= 3:
        return 'Loyal Customers'
    elif row['R_Score'] >= 4 and row['F_Score'] <= 2 and row['M_Score'] >= 3:
        return 'Potential Loyalist'
    elif row['R_Score'] >= 2 and row['F_Score'] >= 4 and row['M_Score'] <= 2:
        return 'Recent Customers'
    elif row['R_Score'] == 1 and row['F_Score'] >= 4 and row['M_Score'] >= 3:
        return 'Promising'
    elif row['R_Score'] == 1 and row['F_Score'] <= 2 and row['M_Score'] <= 2:
        return 'Needs Attention'
    elif row['R_Score'] <= 2 and row['F_Score'] <= 2 and row['M_Score'] <= 2:
        return 'At Risk'
    elif row['R_Score'] == 3 and row['F_Score'] == 1 and row['M_Score'] <= 2:
        return 'About To Sleep'
    elif row['R_Score'] == 2 and row['F_Score'] == 1 and row['M_Score'] <= 2:
        return 'Hibernating'
    elif row['R_Score'] == 1 and row['F_Score'] == 1 and row['M_Score'] == 1:
        return 'Lost'
    else:
        return 'Others'

# Aplicar a função ao DataFrame
rfm['Segment'] = rfm.apply(rfm_segment, axis=1)

# Ordenar por segmento (ou outro critério se preferir)
rfm = rfm.sort_values(by=['R_Score', 'F_Score', 'M_Score'], ascending=[False, False, False])

# Visualizar resultado
rfm.head()

# Salvar resultado se quiser
# rfm.to_csv('rfm_segments.csv', index=False)


,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,Segment
1,12347.0,2,7,4310.00,5,5,5,Champions
15,12362.0,3,10,5226.23,5,5,5,Champions
56,12417.0,3,9,3649.10,5,5,5,Champions
71,12433.0,1,7,13375.87,5,5,5,Champions
75,12437.0,2,18,4951.41,5,5,5,Champions


In [5]:
rfm_quantidade_segmentacao = rfm.groupby("Segment")["CustomerID"].count().reset_index()

rfm_quantidade_segmentacao.columns = ['Segment', 'n_customer']

rfm_quantidade_segmentacao


,Segment,n_customer
0,About To Sleep,123
1,At Risk,268
2,Champions,962
3,Loyal Customers,758
4,Needs Attention,556
5,Others,1467
6,Potential Loyalist,85
7,Promising,46
8,Recent Customers,73


In [8]:
rfm['Segment'].unique()

array(['Champions', 'Loyal Customers', 'Recent Customers', 'Others',
       'Potential Loyalist', 'About To Sleep', 'At Risk', 'Promising',
       'Needs Attention'], dtype=object)

In [ ]:
rfm.to_csv(r'C:\Users\heloi\OneDrive\Desktop\Projeto - Inlytics\dados_temporarios_rfm_csv\rfm_dados_gerais.csv', index=False)

rfm_quantidade_segmentacao.to_csv(r'C:\Users\heloi\OneDrive\Desktop\Projeto - Inlytics\dados_temporarios_rfm_csv\rfm_segmentacao_quantidade.csv', index=False)